In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict

from qiskit_metal.qlibrary.qubits.transmon_cross import TransmonCross
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.core import QRoute, QRoutePoint



design = designs.DesignPlanar()
gui = MetalGUI(design)

design.overwrite_enabled = True

In [3]:
from qiskit_metal.qlibrary.couplers.cap_n_interdigital_tee import CapNInterdigitalTee
from qiskit_metal.qlibrary.core import QComponent

for option in QComponent.default_options:
    print(f'cavity_options.coupler_options.{option}')

cavity_options.coupler_options.pos_x
cavity_options.coupler_options.pos_y
cavity_options.coupler_options.orientation
cavity_options.coupler_options.chip
cavity_options.coupler_options.layer


In [4]:
xmon_options = dict(
    connection_pads=dict(
        readout = dict(connector_location = '90', 
                 connector_type = '0', 
                 claw_length = '215um',
                 ground_spacing = '10um', 
                 claw_gap = '5.1um', 
                 claw_width = '15um'),
    ),
    cross_width = '30um',
    cross_length = '240um',
    cross_gap = '30um',
    aedt_hfss_inductance = 9.686E-9
)

clt_opts = Dict(prime_width = '10um',
                prime_gap = '5.1um',
                second_width = '10um',
                second_gap = '5.1um',
                coupling_space = '7.9um',
                coupling_length = '225um',
                open_termination = False,
                down_length = '50um')

cpw_opts = Dict(pin_inputs = Dict(start_pin = Dict(component = 'clt',
                                                    pin = 'second_end'),
                                   end_pin = Dict(component = 'qubit',
                                                  pin = 'readout')),
                lead = Dict(end_straight = '70um',
                            # start_jogged_extension = jogsS),
                ),
                fillet = '49.9um',
                total_length = '3900um',#'3893.8166um', # '6776um',
                trace_width = '10um',
                meander = Dict(spacing = '100um',
                               asymmetry = '-150um'),
                trace_gap = '5.1um',)


In [5]:
qubit_cavity_opts = Dict(
    qubit_options = xmon_options,
    cavity_options = Dict(
        coupling_type = 'capacitive',
        coupler_options = clt_opts,
        cpw_options = cpw_opts
    )
)

In [6]:
from qubit_cavity import QubitCavity

# xmon = TransmonCross(design, 'qubit', options = xmon_options)
# clt = CoupledLineTee(design, 'clt', clt_opts)
# cpw = RouteMeander(design, 'cpw', options = cpw_opts)
qubitcavity = QubitCavity(design, 'qubit_cavity', options = qubit_cavity_opts)

gui.rebuild()
gui.autoscale()
# gui.zoom_on_components(['cavity_qubit'])

In [7]:
from qiskit_metal.renderers.renderer_ansys_pyaedt.hfss_renderer_eigenmode_aedt import QHFSSEigenmodePyaedt
import pyEPR as epr

In [8]:
em1_aedt = QHFSSEigenmodePyaedt(design, 
                                'test_qubit_cavity_saruman_proj',
                                'test_qubit_cavity_saruman_design',
                                initiate=True)
hfss_aedt = em1_aedt.current_app

pyaedt INFO: using existing logger.


INFO:Global:using existing logger.


pyaedt INFO: Launching PyAEDT outside AEDT with CPython and PythonNET.


INFO:Global:Launching PyAEDT outside AEDT with CPython and PythonNET.


pyaedt INFO: AEDT installation Path D:\Program Files\ANSYS\AnsysEM21.1\Win64.


...

INFO:Global:Design Loaded


pyaedt INFO: Successfully loaded project materials !


INFO:Global:Successfully loaded project materials !


pyaedt INFO: Materials Loaded


INFO:Global:Materials Loaded


In [9]:
def render_design():
    em1_aedt.render_design()

In [10]:
def run_simulation():
    #### Computer's variables
    num_cores=16
    num_gpu=1
    
    #### Setup Name
    setup_name = 'TransmonSetup'
    
    # Change Silicon to Ultracold Silicon
    hfss_aedt.materials['silicon'].permittivity = 11.45
    
    # Add project variables
    add_project_level_vars(em1_aedt)
    
    # Clear setups
    if len(hfss_aedt.setups) != 0:
        hfss_aedt.setups[0].delete()
    
    # Add Setup
    setup_options = dict(
        name= setup_name,
        MinimumFrequency = 3,  # GHz
        NumModes= 1,
        MaxDeltaFreq = 0.1,
        MaximumPasses= 1,
        MinimumPasses= None,
        MinimumConvergedPasses= 2,
        PercentRefinement= 30,
        BasisOrder= None
    )
    
    em1_aedt.add_hfss_em_setup(**setup_options)
    
    hfss_aedt.analyze_setup(name=setup_name, 
                            num_cores=num_cores, 
                            num_gpu=num_gpu)
    
    # Make silicon ultra cold
    hfss_aedt.materials['silicon'].permittivity = 11.45

def add_project_level_vars(renderer):
    """
    Finds all names, inductances, and capacitances of Josephson Junctions rendered into ANSYS.

    Args:
        renderer (QHFSSEigenmodePyaedt)
    """
    # Get all josephson junctions from rendered components table
    geom_table = renderer.path_poly_and_junction_with_valid_comps
    all_jjs = geom_table.loc[geom_table['name'].str.contains('rect_jj')]
    all_jjs = all_jjs.reset_index(drop=True)

    for i, row in all_jjs.iterrows():
        ### Parsing Data ###
        component = str(row['component'])
        name = str(row['name'])
        inductance = row['aedt_hfss_inductance']  # Lj in Henries
        capacitance = row['aedt_hfss_capacitance']  # Cj in Farads

        # Get ANSYS > Model > Sheet corresponding to JJs
        rect_name = 'JJ_rect_Lj_' + component + '_' + name

        # Get ANSYS > Model > Lines corresponding to JJs
        line_name = 'JJ_Lj_' + component + '_' + name + '_'

        ### Appending data ###
        # Add global Lj and Cj variables to ANSYS (for EPR analysis)
        ansys_Lj_name = f'Lj_{i}'
        ansys_Cj_name = f'Cj_{i}'

        renderer.set_variable(ansys_Lj_name, str(inductance * 1E9) + 'nH')
        renderer.set_variable(ansys_Cj_name, str(capacitance * 1E15) + 'fF')

def run_analysis():
    '''
    After finishing simulation, analyze results
    '''
    em1_aedt.run_epr()
    
    # Extract observables of int
    observables = extract_observables(em1_aedt.epr_quantum_analysis)
    return observables



def extract_observables(epra, 
                        cos_trunc: int = 10, 
                        fock_trunc: int = 15):
    '''
    Extract observables from EPR quantum analysis
    
    Args:
        epra (epr.QuantumAnalysis): Raw EPR Quanutm analysis class
        cos_trunc (int, optional): amount of terms to consider in cosine expansion
        fock_trunc (int, optional): size of fock space before truncation
        
    Returns:
        package (dict): simulated observables
    '''
    
    # Extraction of variables
    omegas = epra.get_frequencies()
    chis = epra.get_chis()
    other_data = epra.data
    
    package = dict(qubit_freq_MHz = omegas['0'][0],
                   qubit_anharmonicity_MHz = chis[0][0],
                   other_data = str(other_data))
    
    return package

In [11]:
def run_single_design(): # this will be used as `custom_analysis`

    render_design()
    run_simulation()
    package = run_analysis()
    return package

In [11]:
!pip install C:\Users\lfl\metal-library\.

Processing c:\users\lfl\metal-library
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for metal-library: filename=metal_library-1.0.0-py3-none-any.whl size=404249 sha256=c43b76bf438e3cf030a7ca9425add30b781a8770f17fee6ba53bde8affac8e68
  Stored in directory: c:\users\lfl\appdata\local\pip\cache\wheels\34\e2\4c\a02c623e4f4c556c32c0dcf160e38688f1ca332cdd53acd0b4
Successfully built metal-library
  Attempting uninstall: metal-library
    Found existing installation: metal-library 1.0.0
    Uninstalling metal-library-1.0.0:
      Successfully uninstalled metal-library-1.0.0


In [12]:
from metal_library import QSweeper
sweeper = QSweeper(design)

In [13]:
sweep_parameters = Dict(
  qubit_options = Dict(
    connection_pads = Dict(
      readout = Dict(
        connector_type = '0', 
        claw_length = ['185um', '195um', '205um', '215um'],
        ground_spacing = ['4um', '5um', '7um', '9um', '10um'], 
        claw_gap = ['5.1um'], 
        claw_width = ['10um', '15um'],
        claw_cpw_length=['40um'],
        claw_cpw_width=['10um'],
        connector_location = '0'
        )
      ),
    cross_width = ['30um'],
    cross_length = ['185um', '195um', '205um', '215um', '225um', '235um'],
    cross_gap = ['29um'],
    aedt_hfss_inductance = [9.686E-9, 10.471E-9, 11.268E-9, 12.679-9, 13.816]
    ),
  cavity_options = Dict(
    coupling_type = ['capacitive'],
    coupler_options = Dict(
      coupling_length = ['200um', '225um'],
      coupling_space = ['7um', '7.9um']
    ),
    cpw_options = Dict(total_length = ['3900um', '8150um', '8025um', '3700um', '3550um', '8000um', '3.58mm', '3.5mm', '3.46mm'])
  )
)

my_librarian = sweeper.run_single_component_sweep(component_name='qubit_cavity',
                                                  parameters = sweep_parameters,
                                                  custom_analysis = run_single_design,
                                                  parameters_slice=slice(1,2),
                                                  save_path="./test_qubit_cavity_sweep.csv")

  0%|          | 0/1 [00:00<?, ?it/s]


AttributeError: 'DesignPlanar' object has no attribute 'ls'

In [20]:
metal.__version__

'0.1.5'

In [14]:
design.get_design_name()

'my_design'

In [15]:
dir(design)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__i_am_design__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_assign_name_design',
 '_chips',
 '_components',
 '_delete_all_pins_for_component',
 '_delete_component',
 '_get_new_qcomponent_id',
 '_get_new_qcomponent_name_id',
 '_init_metadata',
 '_metadata',
 '_qcomponent_latest_assigned_id',
 '_qcomponent_latest_name_id',
 '_qgeometry',
 '_qnet',
 '_renderers',
 '_start_renderers',
 '_template_options',
 '_template_renderer_options',
 '_variables',
 'add_chip_info',
 'add_default_data_for_qgeometry_tables',
 'add_dependency',
 'all_component_names_id',
 'build_logs',
 'chips',
 'components',
 'connect_pins',
 'copy_multiple_qcomponents',
 'copy_qcomponent',
 '